# Practice_Sentiment Analysis

비문법/비정형성이 높게 나타나는 커뮤니티 텍스트 감성분석 연습 프로젝트

* 대상 : 에픽세븐 갤러리
* 방법 : 웹크롤링을 통한 기간 지정 텍스트 수집 후 기계학습을 통한 감성어 분석으로 게시글/일자 단위 긍정/부정성 측정


### Roadmap
* LD(Levenshtein Distance) 알고리즘의 작성
   * 두 단어를 동일하게 만들기 위해 수정/연산이 몇 번 필요한지를 계산하여 단위 텍스트 간 유사도 측정

```
LD(a, len_a, b, len_b):
    ### 위 알고리즘에서 len_w는 문자열 w의 길이를
    ### 의미하며 w_i의 형태는 문자열 w의 i번째 문자
    ### 를 의미한다. 위의 알고리즘은 문자열 a를 문자
    ### 열 b와 같도록 하는 거리를 계산하는 알고리즘
    ### 을 의미한다.
    
    if len_a = 0 then return len_b
    if len_b = 0 then return len_a
    
    if a_(len_a - 1) = b_(len_b - 1) then cost = 0
    else cost = 1
    
    return min(LD(a, len_a - 1, b, len_b) + 1,
               LD(a, len_a, b, len_b - 1) + 1,
               LD(a, len_a - 1, b, len_b - 1) + cost)
    
raw algorithm 출처
*** http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.832.2808&rep=rep1&type=pdf
```

* 학습용 텍스트 데이터셋 수집 (긍정/부정성이 강하게 나타나는 문자열 위주 에픽세븐 갤러리 게시글 1천 개)
  * 라이브 커뮤니티 특성상 게시글이 수 초 단위로 생성되므로, 갤러리 1p - 20p (페이지당 게시글 50건)까지를 한번에 저장 후 분석.
* 

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

import re
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [2]:
chromedriver = '/Users/choigww/local/Cellar/chromedriver/chromedriver'
phantomdriver = '/Users/choigww/local/Cellar/phantomjs-2.1.1-macosx/bin/phantomjs'

In [3]:
# 에픽세븐 갤러리 랜딩페이지 (first page)

epic7_gal_default = 'http://gall.dcinside.com/mgallery/board/lists?id=epicseven'
epic7_gal_pageview = 'http://gall.dcinside.com/mgallery/board/lists/?id=epicseven&page='

In [4]:
res = requests.get(epic7_gal_default)
soup = BeautifulSoup(res.content, 'html.parser')

#review_date = soup.select('span.a-size-base.a-color-secondary.review-date')
#review_text= soup.select('div.a-row.a-spacing-medium.review-data > span')

In [8]:
# 1p - 20p까지의 에픽세븐 갤러리 게시글 1000개 저장
epic7_gal_1000_posts = []
for i in range(20): epic7_gal_1000_posts.append(BeautifulSoup(requests.get(epic7_gal_pageview+str(i+1)).content,
                                               'html.parser'))

In [33]:
epic7_gal_1000_posts[1].text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEPIC 7 갤러리\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n\t\tdocument.domain = "dcinside.com";\r\n\t\tvar k_cnt = 0;\r\n\t\tvar _GALLERY_TYPE_ = "M";\r\n\t\n\n\n\n\n\n\n\n\n\r\n      window._taboola = window._taboola || [];\r\n      _taboola.push({category:\'auto\'});\r\n      !function (e, f, u, i) {\r\n        if (!document.getElementById(i)){\r\n          e.async = 1;\r\n          e.src = u;\r\n          e.id = i;\r\n          f.parentNode.insertBefore(e, f);\r\n        }\r\n      }(document.createElement(\'script\'),\r\n      document.getElementsByTagName(\'script\')[0],\r\n      \'//cdn.taboola.com/libtrc/dcinside/loader.js\',\r\n      \'tb_loader_script\');\r\n      if(window.performance && typeof window.performance.mark == \'function\')\r\n      {window.performance.mark(\'tbl_ic\');}\r\n\t\n\n\n\n\n\n\n통합검색 바로가기\n본문영역 바로가기\n상단 메뉴 바로가기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n갤러리 검색\n\n\n\n통합검색\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n갤러리

### HTML structure analysis

BeautifulSoup -> String 자료형 변환 후 기본 구조 (텍스트 타이틀 리스트)

* Start indicator
```
epic 7 갤러리 공지사항\n\n\n솔무향 
```

* end indicator
```
\n\n\n\n\n\n\n\n\n\n\n\n\n\n  var googletag = googletag
```

In [16]:
start_indicator = 'epic 7 갤러리 공지사항\n\n\n솔무향 '
end_indicator = '\n\n\n\n\n\n\n\n\n\n\n\n\n\n  var googletag = googletag'

In [14]:
epic7_gal_1000_posts[0].text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEPIC 7 갤러리\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n\t\tdocument.domain = "dcinside.com";\r\n\t\tvar k_cnt = 0;\r\n\t\tvar _GALLERY_TYPE_ = "M";\r\n\t\n\n\n\n\n\n\n\n\n\r\n      window._taboola = window._taboola || [];\r\n      _taboola.push({category:\'auto\'});\r\n      !function (e, f, u, i) {\r\n        if (!document.getElementById(i)){\r\n          e.async = 1;\r\n          e.src = u;\r\n          e.id = i;\r\n          f.parentNode.insertBefore(e, f);\r\n        }\r\n      }(document.createElement(\'script\'),\r\n      document.getElementsByTagName(\'script\')[0],\r\n      \'//cdn.taboola.com/libtrc/dcinside/loader.js\',\r\n      \'tb_loader_script\');\r\n      if(window.performance && typeof window.performance.mark == \'function\')\r\n      {window.performance.mark(\'tbl_ic\');}\r\n\t\n\n\n\n\n\n\n통합검색 바로가기\n본문영역 바로가기\n상단 메뉴 바로가기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n갤러리 검색\n\n\n\n통합검색\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n갤러리

In [21]:
# 1페이지 게시글 50개의 포스팅 제목 1차 필터링

test = (epic7_gal_1000_posts[0].text).split(start_indicator)[1].split(end_indicator)[0]
test = test[15:]
test

'\n329856\n\n도도미 목소리가 아줌마 목소리는 아닌 듯\n\n\nㅇㅇ(223.33) \n10/25\n1\n0\n\n\n329855\n\n보물섬.jpg\n\n\n라스 \n10/25\n12\n0\n\n\n329854\n\n근데 바라는게 뭐냐ㅋㅋㅋㅋㅋ\n[1] \n\nㅇㅇ(175.223) \n10/25\n20\n0\n\n\n329853\n\n클린갤질 하고싶다\n\n\nㅇㅇ(124.194) \n10/25\n7\n0\n\n\n329852\n\n픽시브 주소 여기.\n[1] \n\n사공이 \n10/25\n33\n0\n\n\n329850\n\n이거 왜이러냐 ;;;\n[1] \n\n개대지 \n10/25\n12\n0\n\n\n329849\n\n도리스쨩이 꿀캐인 이유\n[1] \n\nYawgmoth \n10/25\n20\n0\n\n\n329848\n\n데티 아카 둘다 써본결과\n\n\n육공주니오르 \n10/25\n10\n0\n\n\n329846\n\n이정도면 씹 멍청이 인정하냐?\n\n\nㅇㅇ(219.249) \n10/25\n16\n0\n\n\n329844\n\n600하늘석 박은거 보답받앗다\n[5] \n\nㅇㅇ(115.11) \n10/25\n51\n0\n\n\n329843\n\n지금 도배하는새끼 왜저러는지 알려줌\n[4] \n\n갸. \n10/25\n54\n2\n\n\n329842\n\n62층 진짜 애미가 없네.\n\n\nㅇㅇ(211.176) \n10/25\n2\n0\n\n\n329841\n\n방금 덱조합 문의햇던애인데\n[4] \n\nㅇㅇ(121.140) \n10/25\n18\n0\n\n\n329840\n\n솔무향 완장 내리고 꿀♤때♧지 갤주해라\n\n\n샬롯(223.38) \n10/25\n8\n0\n\n\n329838\n\n실크 화살 고쳤다매ㅋㅋㅋㅋ \n[2] \n\nㅇㅇ(116.124) \n10/25\n59\n1\n\n\n329837\n\n그래도 데티쟝 스작해조서 자힐 잘함\n[4] \n\n크아오아왕 \n10/25\n25\n0\n\n\n329833\n\n완장들 안

In [32]:
(epic7_gal_1000_posts[1].text).split(start_indicator)[1].split(end_indicator)[0]

IndexError: list index out of range

### 게시글 기본정보
`'\n\n'` 문자열 통해 게시글당 3개 단위로 기본정보 출력.
* 1번째 요소 : 게시글 번호 (유니크 id)
  * ex. `\n259601`
* 2번째 요소 : 게시글 제목 
  * ex. `'알바야 너무 티나게 여론몰이하는거 아니냐'`
* 3번째 요소 : 작성자, (ip), 작성월, 작성일, 조회수, 추천수
  * ex. `'\nㅇㅇ(211.253) \n10/16\n2\n0'`

  
#### 게시글 메타데이터 수집 포맷
* 딕셔너리 - {`게시글 번호` : `제목`, `조회수`, `추천수`, `작성월일`}

#### 메타데이터 활용
* 제목으로부터 기본적인 감성 점수 (-1 ~ +1) 계산
* 조회수 및 추천수를 가중치로 부여

In [22]:
test.split('\n\n')[:3]

['\n329856', '도도미 목소리가 아줌마 목소리는 아닌 듯', '\nㅇㅇ(223.33) \n10/25\n1\n0']

In [25]:
test.split('\n\n')[2].split('\n')[1:]

['ㅇㅇ(223.33) ', '10/25', '1', '0']

In [29]:
test.split('\n\n')[2].split('\n')[0] == ''

True

In [36]:
meta_dict = {}
temp_title, temp_nums, temp_id = [], [], []

for i, sp in enumerate(test.split('\n\n')):
    
    print(sp)
    
    if i%3==1:
        if sp.endswith(' '):
            temp_title.append(sp[:-6])
        else:
            temp_title.append(sp)
    
    elif i%3==2:
        #print(sp)
        
        if sp[0] == '':
            nums = sp.split('\n')[1:]
        
        else:
            nums = sp.split('\n')
        
        try:
            if len(nums)==5:
                temp_nums.append((nums[3], nums[4], nums[2]))
            else:
                temp_nums.append((nums[2], nums[3], nums[1]))

        except:
            break
                
    elif i%3==0:
        
        try:
            print(temp_id)
            temp_id.append(int(sp[2:]))
            meta_dict[sp[1:]] = [temp_title[i//3], temp_nums[i//3]]
        
        except:
            meta_dict[temp_id[-1] - 1] = [temp_title[i//3], temp_nums[i//3]]
        


329856
[]


IndexError: list index out of range

In [38]:
meta_dict

{'282428': ['골렘 9단 오토 팁', ('14', '0', '10/18')],
 '282429': ['그래요 내가 마지막 스몰꼬추 루트비히입니', ('31', '0', '10/18')],
 '282430': ['심연 라비도 십거품이', ('41', '0', '10/18')],
 '282431': ['아침부터 지금까지 상급에서 세릴라 안나오는', ('38', '0', '10/18')],
 '282432': ['오구오구 루트비히 우리강아지! 할미가 꼬추좀보자', ('23', '0', '10/18')],
 '282433': ['마야 목소리 깬다;', ('54', '0', '10/18')],
 '282434': ['메갈년들 남캐 존나게 빨잖아', ('25', '0', '10/18')],
 '282435': ['헤이스트 딱 3일안에 재평가 당', ('59', '0', '10/18')],
 '282436': ['캐릭갈아서 인삼 80퍼라도 뽑을수있으면좋겠다', ('20', '0', '10/18')],
 '282437': ['사실상 도적 3대장이 에픽세븐 3대장이', ('53', '0', '10/18')],
 '282438': ['라비 왜하향당한거냐고 하는사람들 라비 하향전에써봄', ('75', '0', '10/18')],
 '282439': ['환불받았다 갈갈꼬접함ㅂ', ('73', '5', '10/18')],
 '282440': ['헤이스트 왜이렇게 거품심하냐 데세라가 넘사벽이면', ('44', '0', '10/18')],
 '282441': ['카마인이 골렘 10단 도는데 라비는 왜', ('21', '0', '10/18')],
 '282442': ['도적 좆망겜', ('4', '0', '10/18')],
 '282443': ['물속 풀속 불속.tx', ('74', '0', '10/18')],
 '282444': ["메갈겜이라 남캐가 사기인건 '당연'하", ('31', '0', '10/18')],
 '282445': ['난 왜이렇게 픽뚫이 잘되지', ('19',

### 함수로 저장하여 복수의 페이지에 있는 게시글 메타정보 크롤링

In [51]:
epic7_gal_1000_posts[1].text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEPIC 7 갤러리\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n\t\tdocument.domain = "dcinside.com";\r\n\t\tvar k_cnt = 0;\r\n\t\tvar _GALLERY_TYPE_ = "M";\r\n\t\n\n\n\n\n\n\n\n\n\r\n      window._taboola = window._taboola || [];\r\n      _taboola.push({category:\'auto\'});\r\n      !function (e, f, u, i) {\r\n        if (!document.getElementById(i)){\r\n          e.async = 1;\r\n          e.src = u;\r\n          e.id = i;\r\n          f.parentNode.insertBefore(e, f);\r\n        }\r\n      }(document.createElement(\'script\'),\r\n      document.getElementsByTagName(\'script\')[0],\r\n      \'//cdn.taboola.com/libtrc/dcinside/loader.js\',\r\n      \'tb_loader_script\');\r\n      if(window.performance && typeof window.performance.mark == \'function\')\r\n      {window.performance.mark(\'tbl_ic\');}\r\n\t\n\n\n\n\n\n\n통합검색 바로가기\n본문영역 바로가기\n상단 메뉴 바로가기\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n갤러리 검색\n\n\n\n통합검색\n\n\n\n\n검색\n\n\n\n\n\n\n\n\n\n\n\n갤러리\n갤로

In [ ]:
start_indicator = '\n09/26\n1934\n3\n\n\n282471\n\n'
end_indicator = '\n\n\n\n\n\n\n \n \n\n\n\n\n\n\n  var googletag = googletag'

In [6]:
def get_epic_gal_meta_infos(url_format, start_idc, end_idc, max_pnum):
    
    # landing_url : 랜딩페이지 url
    # start_idc_land : 게시글 메타정보가 시작되는 문자열 (랜딩페이지)
    # end_idc_land : 게시글 메타정보가 끝나는 문자열 (랜딩페이지)
    # start_idc, end_idc : 게시글 메타정보 시작/종료 문자열 (2페이지부터)
    # max_pnum : 1페이지부터 크롤링하고자 하는 페이지 수
    
    epic7_gal_posts = []
    
    for i in range(max_pnum): 
        epic7_gal_posts.append(BeautifulSoup(requests.get(url_format+str(i+1)).content,\
                                             'html.parser'))
    
    meta_dict = {}
    temp_title, temp_metas = [], []
    temp_num_comments, current_post_num = 0, 0
    
    for p in range(max_pnum):
        
        # 첫번째 페이지 제외.
        if p == 0:
            continue
        
        else:
            test__ = (epic7_gal_posts[p].text).split(start_idc)[1].split(end_idc)[0]
            test = (epic7_gal_posts[p].text).split(start_idc)[1].split(end_idc)[0]
        
        print(repr(test__))
        print('')
        
        for i, sp in enumerate(test.split('\n\n')):
            
            # post title
            if i%3==1:
                
                is_commented = re.findall(r'\n\[[0-9]+\]', sp)
                
                if is_commented:
                    #print(sp)
                    temp_title = sp[:-len(is_commented[0])].strip(' ').strip('\n')
                    
                    #print(is_commented[0].strip(' ').strip('\n'))
                    
                    #temp_num_comments = int(sp[-len(re.findall(r'\n\[[0-9]+\]', sp)[0]):].strip('[').strip(']'))
                    temp_num_comments = int(is_commented[0].strip(' ').strip('\n')[1:-1])
                    #print(temp_num_comments)
                    
                else:
                    temp_title = sp.strip(' ').strip('\n')
                    
                meta_dict[current_post_num].append(temp_title)
            
            # post meta data (writer name, date, views, recommendations)
            # format example: '\n283186', '다음 6성 헤이스트 ㄱ?\n[7] ', '떠그 \n10/18\n141\n0'
            elif i%3==2:
                
                metas = sp.split('\n')
                temp_metas = [metas[2].strip(' '), 
                              metas[3].strip(' '), 
                              metas[1].strip(' ')]
                
                temp_metas.append(temp_num_comments)
                meta_dict[current_post_num].append(temp_metas)
            
            # post number + adding meta data
            elif i%3==0:
                #temp_id.append(sp.strip('\n'))
                
                current_post_num = int(sp.strip('\n'))
                
                if temp_num_comments:
                    temp_metas.extend([temp_num_comments])
                    meta_dict[current_post_num] = []
                
                else:
                    meta_dict[current_post_num] = []
                    
                

        #print(meta_dict)
        
        # 임시 정보저장용 변수 초기화 / 증가
        temp_title, temp_metas, temp_num_comments = [], [], 0
        current_post_num += 1
                
    return meta_dict

In [7]:
get_epic_gal_meta_infos('http://gall.dcinside.com/mgallery/board/lists/?id=epicseven&page=',
                        'start', '\n\n\n\n\n\n\n', 3)

KeyboardInterrupt: 

In [ ]:
'#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr:nth-child(3) > td.gall_tit.ub-word > a:nth-child(1)'